In [2]:
from urllib.request import urlopen
import ssl

context = ssl._create_unverified_context()
html = urlopen('https://pythonscraping.com/pages/page1.html', context = context)
print(html.read())

# 이렇게만 하면, 에러가 뜨는 것을 확인할 수 있는데, 책에 http였던 링크가
# https로 변해서 그렇다. https를 파싱하려면 'urllib'패키지 자체가 ssl 인증을 요구하므로
# 그 처리까지 해주어야 한다. 그 처리로 context를 반영했다고 보면 된다.

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


In [3]:
#1.2.2 BeautifulSoup 실행

from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://pythonscraping.com/pages/page1.html', context = context)
bs = BeautifulSoup(html.read(), 'html.parser') # 뷰티풀소프 클래스로 bs라는 객체 생성
print(bs.h1) # h1 을 추출해주는 것.

<h1>An Interesting Title</h1>


In [13]:
#1.2.3 신뢰할 수 있는 연결과 예외 처리

#. 페이지를 가져올 수 없는 경우에 대한 예외처리
"""
웹에서 생길 수 있는 문제는 크게 두 가지이다. 
첫째, HTTP 에러_ 페이지를 찾을 수 없거나, URL 해석에서 에러가 생긴 경우 (ex. 404, 500 등 서버 등의 문제)
둘째, URL 에러_ 서버를 찾을 수 없는 경우
"""

from urllib.request import urlopen
from urllib.request import HTTPError

try:
    html = urlopen('https://pythonscraping.com/pages/page1.html', context = context)
except HTTPError as e:
    print(e)
except URLError as e:
    print('The server could not be found!')
else:
    print('It Worked!')
    
# try를 할 때, 문제가 생기면 except로 / 그렇지 않으면 else로 간다.

It Worked!


In [8]:
#. 페이지 콘텐츠가 예상과 전혀 다른 경우에 대한 예외 처리

print(bs.nonExistentTag) # None 객체를 반환하도록 한 것으로, 실제로는 존재하지 않는 태그이다.
# 여기에 someTag를 출력해보자.

# print(bs.nonExistentTag.someTag) 

"""
자 보자. bs는 하나의 클래스 객체이고 올바르게 현재 어떤 http 주소의 html 코드가 담겨있는 상태이다.
이때, '태그'기능을 살펴보자.

앞서 보았던 'bs.h1'은 bs라는 객체에서 h1이라는 태그를 추출해내는 코드이다.
h1은 실제로 html 상에 존재하는 태그였기 때문에 '<h1>An Interesting Title</h1>'라는 결과를 출력해냈다.

하지만, 현재 작성한 윗줄의 코드에서 nonExistentTag는 존재하지 않는 가상의 태그이다.
이러한 경우 뷰티풀솦은 None을 반환한다.(태그가 실제로 존재하지 않는 경우)

사실 단순히 이 경우에는 상관이 없지만, 
그러한 None 객체 자체에 태그가 있다고 가정하고 그 태그에 접근하려 하면 AttributeError가 일어난다.

가장 아래 코드에 주석처리를 빼고 입력하면 확인할 수 있다.
"""
print("---")

#. 이런 상황에 어떻게 대응해야 할까?

try:
    badContent = bs.nonExistingTag.anotherTag
except AttributeError as e:  # nonExistingTag가 없는 경우
    print("Tag was not found")
else:
    if badContent == None:
        print("Tag was not found")   # anotherTag가 없는 경우.
    else:
        print(badContent)

None


"\n자 보자. bs는 하나의 클래스 객체이고 올바르게 현재 어떤 http 주소의 html 코드가 담겨있는 상태이다.\n이때, '태그'기능을 살펴보자.\n\n앞서 보았던 'bs.h1'은 bs라는 객체에서 h1이라는 태그를 추출해내는 코드이다.\nh1은 실제로 html 상에 존재하는 태그였기 때문에 '<h1>An Interesting Title</h1>'라는 결과를 출력해냈다.\n\n하지만, 현재 작성한 윗줄의 코드에서 nonExistentTag는 존재하지 않는 가상의 태그이다.\n이러한 경우 뷰티풀솦은 None을 반환한다.(태그가 실제로 존재하지 않는 경우)\n\n사실 단순히 이 경우에는 상관이 없지만, \n그러한 None 객체 자체에 태그가 있다고 가정하고 그 태그에 접근하려 하면 AttributeError가 일어난다.\n\n가장 아래 코드에 주석처리를 빼고 입력하면 확인할 수 있다.\n"

In [14]:
# 이렇게 가능한 에러를 모두 체크하여 코드를 수정해보자.

from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import ssl # 개인적인 에러처리 (책과 별개의)

context = ssl._create_unverified_context()

def getTitle(url):
    try:
        html = urlopen(url,context = context)
    except HTTPError as e:
        return None
    except URLError as e:  # 책에는 URLError 부분이 안써있던데, 똑같이 None을 반환하는 것일까? 
        return None        # 우선 쭉 따라가보자.

    try:
        bs = BeautifulSoup(html.read(), 'html.parser')
        title = bs.body.h1
    except AttributeError as e:
        return None
    
    return title

title = getTitle('https://www.pythonscraping.com/pages/page1.html')

if title == None:
    print('Title could not be found')
else:
    print(title)
    
"""
이와 같이 getTitle이나 getSiteHTML과 같은 범용 함수를 만들고 여기에 예외 처리를 철저하게 만들어두면
빠르고 믿을 수 있는 웹스크레이퍼를 예외처리를 일일이 작성하며 번거롭지 않게 만들 수 있다.
"""

<h1>An Interesting Title</h1>


'\n이와 같이 getTitle이나 getSiteHTML과 같은 범용 함수를 만들고 여기에 예외 처리를 철저하게 만들어두면\n빠르고 믿을 수 있는 웹스크레이퍼를 예외처리를 일일이 작성하며 번거롭지 않게 만들 수 있다.\n'